In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [5]:
df = pd.read_csv("UNSW_NB15_training-set.csv")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82332 entries, 0 to 82331
Data columns (total 45 columns):
id                   82332 non-null int64
dur                  82332 non-null float64
proto                82332 non-null object
service              82332 non-null object
state                82332 non-null object
spkts                82332 non-null int64
dpkts                82332 non-null int64
sbytes               82332 non-null int64
dbytes               82332 non-null int64
rate                 82332 non-null float64
sttl                 82332 non-null int64
dttl                 82332 non-null int64
sload                82332 non-null float64
dload                82332 non-null float64
sloss                82332 non-null int64
dloss                82332 non-null int64
sinpkt               82332 non-null float64
dinpkt               82332 non-null float64
sjit                 82332 non-null float64
djit                 82332 non-null float64
swin                 82332 non-n

In [7]:
cats = df.drop('class',axis=1).select_dtypes(include=['object']).copy()

KeyError: "['class'] not found in axis"

In [8]:
t = cats.columns
for col in t:
    cats[col] = cats[col].astype('category')

NameError: name 'cats' is not defined

In [ ]:
cats.info()

In [ ]:
for col in cats.columns:
    cats[col] = cats[col].cat.codes

In [ ]:
data = df.drop(cats.columns, axis = 1)

In [ ]:
dataset = pd.concat([data,cats],axis = 1)

In [ ]:
xnorm = dataset.drop('label', axis = 1)
ynorm = dataset['class']

In [22]:
main_df = dataset

In [24]:
main_df['label'] = ynorm

In [25]:
main_df.to_csv('network data.csv')

In [11]:
from sklearn.model_selection import train_test_split as tts

In [12]:
import pyspark.sql as sql

In [14]:
from pyspark import SparkContext
sc = SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-13-1470eb07d84e>:2 

In [16]:
sqlContext = sql.SQLContext(sc)

In [17]:
spark_df = sqlContext.createDataFrame(dataset)

In [20]:
for row in spark_df.take(2):
    print(row)

Row(id=1, dur=1.1e-05, spkts=2, dpkts=0, sbytes=496, dbytes=0, rate=90909.0902, sttl=254, dttl=0, sload=180363632.0, dload=0.0, sloss=0, dloss=0, sinpkt=0.011000000000000001, dinpkt=0.0, sjit=0.0, djit=0.0, swin=0, stcpb=0, dtcpb=0, dwin=0, tcprtt=0.0, synack=0.0, ackdat=0.0, smean=248, dmean=0, trans_depth=0, response_body_len=0, ct_srv_src=2, ct_state_ttl=2, ct_dst_ltm=1, ct_src_dport_ltm=1, ct_dst_sport_ltm=1, ct_dst_src_ltm=2, is_ftp_login=0, ct_ftp_cmd=0, ct_flw_http_mthd=0, ct_src_ltm=1, ct_srv_dst=2, is_sm_ips_ports=0, label=0, proto=117, service=0, state=4, attack_cat=6)
Row(id=2, dur=8e-06, spkts=2, dpkts=0, sbytes=1762, dbytes=0, rate=125000.0003, sttl=254, dttl=0, sload=881000000.0, dload=0.0, sloss=0, dloss=0, sinpkt=0.008, dinpkt=0.0, sjit=0.0, djit=0.0, swin=0, stcpb=0, dtcpb=0, dwin=0, tcprtt=0.0, synack=0.0, ackdat=0.0, smean=881, dmean=0, trans_depth=0, response_body_len=0, ct_srv_src=2, ct_state_ttl=2, ct_dst_ltm=1, ct_src_dport_ltm=1, ct_dst_sport_ltm=1, ct_dst_src_l

In [26]:
from pyspark.sql import SparkSession

In [27]:
spark = SparkSession.builder.appName('ml-bank').getOrCreate()

In [88]:
net_data = spark.read.csv('network data - Copy.csv', header = True, inferSchema = True)
net_data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- dur: double (nullable = true)
 |-- spkts: integer (nullable = true)
 |-- dpkts: integer (nullable = true)
 |-- sbytes: integer (nullable = true)
 |-- dbytes: integer (nullable = true)
 |-- rate: double (nullable = true)
 |-- sttl: integer (nullable = true)
 |-- dttl: integer (nullable = true)
 |-- sload: double (nullable = true)
 |-- dload: double (nullable = true)
 |-- sloss: integer (nullable = true)
 |-- dloss: integer (nullable = true)
 |-- sinpkt: double (nullable = true)
 |-- dinpkt: double (nullable = true)
 |-- sjit: double (nullable = true)
 |-- djit: double (nullable = true)
 |-- swin: integer (nullable = true)
 |-- stcpb: long (nullable = true)
 |-- dtcpb: long (nullable = true)
 |-- dwin: integer (nullable = true)
 |-- tcprtt: double (nullable = true)
 |-- synack: double (nullable = true)
 |-- ackdat: double (nullable = true)
 |-- smean: integer (nullable = true)
 |-- dmean: integer (nullable = true)
 |-- trans_depth: integer (nu

In [89]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

In [90]:
categoricalColumns = list(cats.columns)
stages = []


In [91]:
categoricalColumns.remove('proto')

In [92]:
categoricalColumns

['service', 'state', 'attack_cat']

In [93]:
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

In [94]:
label_stringIdx = StringIndexer(inputCol = 'class', outputCol = 'label')
stages += [label_stringIdx]

In [95]:
numericCols = list(dataset.drop(cats.columns, axis=1))

In [99]:
numericCols.remove('class')

ValueError: list.remove(x): x not in list

In [100]:
numericCols.append('proto')

In [101]:
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [102]:
from pyspark.ml import Pipeline

In [106]:
cols = list(pd.read_csv('network data - Copy.csv').columns)

In [108]:
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(net_data)
df = pipelineModel.transform(net_data)
selectedCols = ['label', 'features'] + cols
net_data = net_data.select(selectedCols)
net_data.printSchema()

AnalysisException: "cannot resolve '`label`' given input columns: [sload, tcprtt, ct_dst_ltm, dbytes, ct_srv_dst, attack_cat, response_body_len, ct_ftp_cmd, dur, _c40, trans_depth, dwin, sjit, ackdat, ct_dst_src_ltm, class, sloss, dloss, state, dpkts, id, is_sm_ips_ports, dtcpb, dttl, ct_flw_http_mthd, service, spkts, is_ftp_login, proto, dload, ct_src_ltm, sbytes, dinpkt, dmean, ct_src_dport_ltm, ct_dst_sport_ltm, smean, stcpb, ct_srv_src, swin, sttl, sinpkt, ct_state_ttl, djit, synack, rate];;\n'Project ['label, 'features, id#3709, dur#3710, spkts#3711, dpkts#3712, sbytes#3713, dbytes#3714, rate#3715, sttl#3716, dttl#3717, sload#3718, dload#3719, sloss#3720, dloss#3721, sinpkt#3722, dinpkt#3723, sjit#3724, djit#3725, swin#3726, stcpb#3727L, dtcpb#3728L, dwin#3729, tcprtt#3730, ... 24 more fields]\n+- Relation[id#3709,dur#3710,spkts#3711,dpkts#3712,sbytes#3713,dbytes#3714,rate#3715,sttl#3716,dttl#3717,sload#3718,dload#3719,sloss#3720,dloss#3721,sinpkt#3722,dinpkt#3723,sjit#3724,djit#3